In [2]:
import numpy as np
import pandas as pd

result = pd.read_csv('sample/results.csv', encoding='cp949')
result

,파일명,선종,상태,위치
0,1.jpg,범장망,표류,LAT N33.22.872 LON E124.5.074
1,2.jpg,범장망,이동,LAT N32.48.241 LON E124.38.553
2,3.jpg,범장망,조업,LAT N32.55.893 LON E124.33.731
3,4.jpg,타망,이동,LAT N35.40.891 LON E124.25.481
4,5.jpg,타망,이동,LAT N32.23.668 LON E125.33.053
5,6.jpg,타망,이동,LAT N32.59.352 LON E124.29.609
6,7.jpg,유망,조업,LAT N34.54.232 LON E124.42.394
7,8.jpg,유망,조업,LAT N34.41.829 LON E124.20.120
8,9.jpg,유망,표류,LAT N36.56.279 LON E124.39.427
9,10.jpg,안강망,조업,LAT N34.15.543 LON E125.41.175


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# 사전 학습된 ResNet50 모델 로드 (ImageNet 데이터셋으로 학습됨)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 마지막 레이어를 교체해 새로운 모델 구성
x = base_model.output
x = GlobalAveragePooling2D()(x)  # 전역 평균 풀링
x = Dense(1024, activation='relu')(x)  # 완전 연결 레이어
predictions = Dense(2, activation='softmax')(x)  # 클래스 개수에 맞춘 출력 레이어

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 사전 학습된 레이어는 학습되지 않도록 고정
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 데이터셋 전처리 (ImageDataGenerator를 사용하여 이미지 증식)
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
    'sample', target_size=(224, 224), batch_size=32, class_mode='categorical'
)

# 모델 학습
model.fit(train_generator, steps_per_epoch=100, epochs=10)

# 학습된 모델 저장  
model.save('resnet50_finetuned.h5')


Found 12 images belonging to 2 classes.
Epoch 1/10


c:\Users\kangdongwoo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 5s 318us/step - accuracy: 0.5833 - loss: 0.6930
Epoch 2/10


c:\Users\kangdongwoo\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 87us/step - accuracy: 0.5000 - loss: 2.1244  
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 61us/step - accuracy: 0.5000 - loss: 0.6736  
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 71us/step - accuracy: 0.5000 - loss: 1.6367  
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 81us/step - accuracy: 0.5000 - loss: 1.5743  
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 71us/step - accuracy: 0.5000 - loss: 0.8777  
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 81us/step - accuracy: 0.5000 - loss: 0.7126  
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 76us/step - accuracy: 0.5000 - loss: 1.1412  
Epoch 9/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 108us/step - accuracy: 0.5000 - loss: 1.1376 
Epoch 10/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 81us/step - accuracy: 0.5000 - loss: 0.8016  


In [4]:
import folium

# 좌표 데이터 (위도, 경도)
locations = [
    {"name": "차", "lat": 32 + 11/60, "lon": 125 + 25/60},
    {"name": "카", "lat": 33 + 20/60, "lon": 124 + 8/60},
    {"name": "타", "lat": 34 + 0/60, "lon": 124 + 0/60 + 30/3600},
    {"name": "파", "lat": 35 + 0/60, "lon": 124 + 7/60 + 30/3600},
    {"name": "하", "lat": 35 + 30/60, "lon": 124 + 30/60},
    {"name": "거", "lat": 36 + 45/60, "lon": 124 + 30/60},
    {"name": "너", "lat": 37 + 0/60, "lon": 124 + 20/60},
    {"name": "바", "lat": 32 + 11/60, "lon": 126 + 45/60},
]

# 초기 지도의 중심 좌표 (평균 값 사용)
center_lat = sum([loc["lat"] for loc in locations]) / len(locations)
center_lon = sum([loc["lon"] for loc in locations]) / len(locations)

# 지도 생성
mymap = folium.Map(location=[center_lat, center_lon], zoom_start=7)

# 각 위치에 마커 추가
for loc in locations:
    folium.Marker(
        location=[loc["lat"], loc["lon"]],
        popup=loc["name"]
    ).add_to(mymap)

# 지도 저장
mymap.save("map.html")

# 지도를 출력하고 싶다면, 주피터 노트북 환경에서는 다음 코드로 출력 가능
mymap


In [7]:
import re
from shapely.geometry import Point, Polygon

# 좌표 변환 함수 (DMS -> Decimal)
def dms_to_decimal(degrees, minutes_seconds):
    minutes = float(minutes_seconds) / 60
    return degrees + minutes

# 좌표를 추출하는 함수 (데이터프레임의 특정 형식에서)
def extract_coordinates(df):
    # 위도 N 32° 55.893"
    lat_degree = int(df['TLat'][1][:-1])  # 위도는 N이므로 숫자만 추출 (32)
    lat_minutes_seconds = float(df['TLat'][2][:-1])  # 55.893"에서 숫자만 추출

    # 경도 E 124° 33.731"
    lon_degree = int(df['TLon'][1][:-1])  # 경도는 E이므로 숫자만 추출 (124)
    lon_minutes_seconds = float(df['TLon'][2][:-1])  # 33.731"에서 숫자만 추출

    # 위도와 경도를 소수점 형식으로 변환
    latitude = dms_to_decimal(lat_degree, lat_minutes_seconds)
    longitude = dms_to_decimal(lon_degree, lon_minutes_seconds)

    return latitude, longitude

# 예시 데이터 (입력 형식에 맞게)
data = {
    'TLat': ['N', '32°', '55.893"'],
    'TLon': ['E', '124°', '33.731"']
}

# 위도와 경도 추출
latitude, longitude = extract_coordinates(data)

print(f"변환된 위도: {latitude}")
print(f"변환된 경도: {longitude}")

# 한중 잠정조치수역 동쪽 경계선 좌표 (위도, 경도)
boundary_coords = [
    (37 + 00/60, 124 + 20/60),  # 너 - A16
    (36 + 45/60, 124 + 30/60),  # 거 - A15
    (35 + 30/60, 124 + 30/60),  # 하 - A14
    (35 + 00/60, 124 + 7.5/60), # 파 - A13
    (34 + 00/60, 124 + 0.5/60), # 타 - A12
    (33 + 20/60, 124 + 8/60),   # 카 - A11
    (32 + 11/60, 125 + 25/60),  # 차 - A10
    (32 + 11/60, 126 + 45/60),  # 바 - K6
    (32 + 40/60, 127 + 0/60)    # 사 - K7
]

# 경계선을 폴리곤으로 정의
boundary_polygon = Polygon(boundary_coords)

# 선박의 위치를 변환된 좌표로 정의
ship_location = Point(latitude, longitude)

# 선박이 경계선 안에 있는지 확인
if boundary_polygon.contains(ship_location):
    print("선박이 한국 어선 구역에 진입하지 않았습니다.")
else:
    print("선박이 한국 어선 구역에 진입했습니다.")


변환된 위도: 32.93155
변환된 경도: 124.56218333333334
선박이 한국 어선 구역에 진입했습니다.


In [15]:
import easyocr
import cv2
import os

# EasyOCR Reader 초기화 (한국어 및 영어 설정 가능)
reader = easyocr.Reader(['ko', 'en'])

# 이미지 파일이 저장된 디렉터리 경로
image_dir = 'sample/class2/'  # 이미지들이 저장된 디렉터리

# 디렉터리 내 모든 파일에 대해 반복
for image_file in os.listdir(image_dir):
    # 이미지 파일 경로
    image_path = os.path.join(image_dir, image_file)

    # OpenCV로 이미지 불러오기
    image = cv2.imread(image_path)
    
    if image is None:
        print(f"Skipping {image_file}, not a valid image.")
        continue
    
    # 이미지의 텍스트 추출
    result = reader.readtext(image)

    # 이미지 파일 이름 출력
    print(f"\nExtracting text from {image_file}:")

    # 결과 출력
    for (bbox, text, prob) in result:
        print(f"Detected Text: {text}, Confidence: {prob}")



Extracting text from 10.jpg:
Detected Text: 30, Confidence: 0.9997117277275431
Detected Text: 120, Confidence: 0.9791519045829773
Detected Text: 180, Confidence: 0.7448068081710226
Detected Text: 90, Confidence: 0.9992421994962192
Detected Text: 90, Confidence: 0.9995127329155238
Detected Text: 180, Confidence: 0.7106501194236672
Detected Text: TLat N 340 15.543' TLon E 1255 41.175' Alt, Confidence: 0.5493440749276146
Detected Text: ofs Rng: 3923m Ins Nav HdgUncert=0.05, Confidence: 0.6745346473310145

Extracting text from 11.jpg:
Detected Text: 150, Confidence: 0.999989263227318
Detected Text: 210, Confidence: 0.9997344070617572
Detected Text: 240, Confidence: 0.9837106466293335
Detected Text: 300, Confidence: 0.999944939790755
Detected Text: 1 , Confidence: 0.6166490329091778
Detected Text: 120, Confidence: 0.8210821944384014
Detected Text: 1an, Confidence: 0.9254461125002311
Detected Text: u, Confidence: 0.005190950949274742
Detected Text: 90, Confidence: 0.9585296555875843
Detecte

In [14]:
import easyocr
import cv2
import os
import re
import csv

# EasyOCR Reader 초기화 (한국어 및 영어 설정 가능)
reader = easyocr.Reader(['ko', 'en'])

# 이미지 파일이 저장된 디렉터리 경로
image_dir = 'sample/class2/'  # 이미지들이 저장된 디렉터리

# 저장할 데이터를 담을 리스트
filtered_texts = []

# 정규 표현식을 사용해 'TLat N ... TLon E ... Alt' 형식의 텍스트 필터링
pattern = r'TLat N \d{1,3} \d{1,2}\.\d{1,3}\' TLon E \d{1,3} \d{1,2}\.\d{1,3}\' Alt'

# 디렉터리 내 모든 파일에 대해 반복
for image_file in os.listdir(image_dir):
    # 이미지 파일 경로
    image_path = os.path.join(image_dir, image_file)

    # OpenCV로 이미지 불러오기
    image = cv2.imread(image_path)
    
    if image is None:
        print(f"Skipping {image_file}, not a valid image.")
        continue

    # 이미지의 텍스트 추출
    result = reader.readtext(image)

    # 이미지 파일 이름 출력
    print(f"\nExtracting text from {image_file}:")

    # 각 추출된 텍스트에서 정규 표현식으로 필터링
    for (bbox, text, prob) in result:
        match = re.search(pattern, text)
        if match:
            filtered_texts.append([image_file, match.group()])  # 이미지 파일명과 함께 저장
            print(f"Filtered Text: {match.group()}")

# 필터된 데이터를 CSV 파일로 저장
with open('filtered_texts.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Image File', 'Detected Text'])  # 헤더 작성
    writer.writerows(filtered_texts)  # 필터된 데이터 작성

print("Filtered texts saved to 'filtered_texts.csv'")



Extracting text from 10.jpg:

Extracting text from 11.jpg:

Extracting text from 12.jpg:

Extracting text from 7.jpg:

Extracting text from 8.jpg:

Extracting text from 9.jpg:
Filtered texts saved to 'filtered_texts.csv'
